<h2>Library</h2>

In [607]:
import pandas as pd
import numpy as np
import warnings

import plotly.graph_objects as go
import plotly.io as pio


warnings.filterwarnings('ignore')

<h2>Function</h2>

In [608]:
def format_value(value):
    if value >= 1000000:
        return f'{value/1000000:.1f}M'
    elif value >= 1000:
        return f'{value/1000:.1f}K'
    else:
        return f'{value:.0f}'

In [609]:
def generate_gray_tones(num_colors):
    # Define the range of gray values (0 = black, 255 = white)
    min_gray = 120
    max_gray = 220

    # Calculate the step size to evenly distribute the tones
    step = (max_gray - min_gray) // (num_colors - 1)

    # Generate a list of random gray colors
    gray_colors = []
    for _ in range(num_colors):
        gray_value = min_gray + step * _
        gray_color = "#{:02X}{:02X}{:02X}".format(gray_value, gray_value, gray_value)
        gray_colors.append(gray_color)

    # Shuffle the list to randomize the order
    #random.shuffle(gray_colors)
    
    return gray_colors

In [610]:
def bu_audiencetype_vehicle_sankey_plot(df, value_column, by_Dell, title):
    
    if by_Dell:
        df = df[(df['Segment ID'] != 0) & (df['Segment ID'] != '0')]
        
    if value_column == 'Segment ID':
        df_lvl1 = df.groupby(['BU', 'Audience Type'])[value_column].nunique().reset_index()
        df_lvl2 = df.groupby(['Audience Type', 'Display Dell Vehicle Mapped'])[value_column].nunique().reset_index()
    else:
        df_lvl1 = df.groupby(['BU', 'Audience Type'])[value_column].sum().reset_index()
        df_lvl2 = df.groupby(['Audience Type', 'Display Dell Vehicle Mapped'])[value_column].sum().reset_index()

    df_lvl1['Level'] = 1
    df_lvl2['Level'] = 2

        
    df_lvl1.rename(columns={'BU':'Source', 'Audience Type':'Target', value_column:'Value'}, inplace=True)
    df_lvl2.rename(columns={'Audience Type':'Source', 'Display Dell Vehicle Mapped':'Target', value_column:'Value'}, inplace=True)

    # Concatenate the dataframes into one
    concat_df = pd.concat([df_lvl1[['Source', 'Target', 'Value', 'Level']],
                           df_lvl2[['Source','Target','Value', 'Level']]],
                          ignore_index=True)

    unique_values = pd.unique(concat_df[['Source', 'Target']].values.ravel('K'))

    mapping_df = pd.DataFrame({'Label': unique_values.tolist(),
                               'Value': [i for i in range(len(unique_values))]})

    # Replace values based on the mappings
    concat_df['Source'] = concat_df['Source'].replace(mapping_df.set_index('Label')['Value'])
    concat_df['Target'] = concat_df['Target'].replace(mapping_df.set_index('Label')['Value'])
    
    
    ##########
    level_list = concat_df['Level'].unique().tolist()

    if len(level_list) > 1:
        level_list_source = level_list[:-1]
        level_list_target = [level_list[-1]]  # Wrap the last level in a list
    else:
        level_list_source, level_list_target = [1], [1]


    df_source = concat_df[concat_df['Level'].isin(level_list_source)].groupby(['Source','Level'])['Value'].sum().reset_index()
    total_sum_source = df_source[df_source['Level'] == level_list[0]]['Value'].sum()
    df_source['Percentage'] = (df_source['Value'] / total_sum_source) * 100

    df_target = concat_df[concat_df['Level'].isin(level_list_target)].groupby(['Target','Level'])['Value'].sum().reset_index()
    total_sum_source = df_target[df_target['Level'].isin(level_list_target)]['Value'].sum()
    df_target['Percentage'] = (df_target['Value'] / total_sum_source) * 100


    df_source.rename(columns={'Value':'Total','Source':'Value'}, inplace=True)
    df_target.rename(columns={'Value':'Total','Target':'Value'}, inplace=True)

    totals_df = pd.concat([df_source, df_target])

    merged_df = mapping_df.merge(totals_df, on='Value', how='left')

    merged_df['Label_x'] = merged_df['Label'] + "<br>" + merged_df['Percentage'].apply(lambda x: f'{x:.1f}%') + " (" + merged_df['Total'].apply(format_value) + ")"
############

    

    # Your existing data
    source = concat_df['Source'].values.tolist()
    target = concat_df['Target'].values.tolist()
    value = concat_df['Value'].values.tolist()
    labels = merged_df['Label_x'].values.tolist()

    # List of colors for each link based on some condition or data
    colors = generate_gray_tones(len(source))

    # Create links with specified colors
    link = dict(source=source, target=target, value=value, color=colors)

    # Create nodes
    node = dict(label=labels, pad=30, thickness=20)

    # Create a Sankey object
    chart = go.Sankey(link=link, node=node, arrangement="snap")

    # Build a figure
    fig = go.Figure(chart)

    # Add a title to the figure
    fig.update_layout(title_text=title)

    # Show the figure
    fig.show()
    
    #html_file_path = 'html/' + title + ".html"
    #pio.write_html(fig, file=html_file_path)


In [ ]:
def audiencetype_vehicles_sankey_plot(df, value_column, by_Dell, title):
    
    if by_Dell:
        df = df[(df['Segment ID'] != 0) & (df['Segment ID'] != '0')]
    
    if value_column == 'Segment ID':
        df_lvl1 = df.groupby(['Audience Type', 'Display Dell Vehicle Mapped'])[value_column].nunique().reset_index()
    else:
        df_lvl1 = df.groupby(['Audience Type', 'Display Dell Vehicle Mapped'])[value_column].sum().reset_index()

    df_lvl1.rename(columns={'Audience Type':'Source', 'Display Dell Vehicle Mapped':'Target', value_column:'Value'}, inplace=True)

    unique_values = pd.unique(df_lvl1[['Source', 'Target']].values.ravel('K'))

    mapping_df = pd.DataFrame({'Label': unique_values.tolist()
                                              , 'Value': [i for i in range(len(unique_values))]})

    # Replace values based on the mappings
    df_lvl1['Source'] = df_lvl1['Source'].replace(mapping_df.set_index('Label')['Value'])
    df_lvl1['Target'] = df_lvl1['Target'].replace(mapping_df.set_index('Label')['Value'])

    # Contact the dataframes in just one
    concat_df = pd.concat([
                           df_lvl1[['Source','Target','Value']]
                        ], ignore_index=True)
    
    ##########
    df_source = concat_df.groupby('Source')['Value'].sum().reset_index()
    total_sum_source = df_source['Value'].sum()
    df_source['Percentage'] = (df_source['Value'] / total_sum_source) * 100
    df_source.rename(columns={'Value':'Total','Source':'Value'}, inplace=True)

    df_target = concat_df.groupby('Target')['Value'].sum().reset_index()
    total_sum_target = df_target['Value'].sum()
    df_target['Percentage'] = (df_target['Value'] / total_sum_target) * 100
    df_target.rename(columns={'Value':'Total','Target':'Value'}, inplace=True)

    totals_df = pd.concat([df_source, df_target])

    merged_df = mapping_df.merge(totals_df, on='Value', how='left')

    merged_df['Label_x'] = merged_df['Label'] + "<br>" + merged_df['Percentage'].apply(lambda x: f'{x:.1f}%') + " (" + merged_df['Total'].apply(format_value) + ")"
    ############

    
    
    # Sample data
    source = df_lvl1['Source'].values.tolist()
    target = df_lvl1['Target'].values.tolist()
    value = df_lvl1['Value'].values.tolist()
    labels = merged_df['Label_x'].values.tolist()

    # List of colors for each link based on some condition or data
    colors = generate_gray_tones(len(source))

    # Create links
    link = dict(source=source, target=target, value=value, color=colors)

    # Create nodes
    node = dict(label=labels, pad=30, thickness=20)

    # Create a Sankey object
    chart = go.Sankey(link=link, node=node, arrangement="snap")

    # Build a figure
    fig = go.Figure(chart)
    
    # Add a title to the figure
    fig.update_layout(title_text=title)

    fig.show()
    
    #html_file_path = 'html/' + title + ".html"
    #pio.write_html(fig, file=html_file_path)

In [ ]:
def funnel_sankey_plot(df, value_column, by_Dell, title):
    
    if by_Dell:
        df = df[(df['Segment ID'] != 0) & (df['Segment ID'] != '0')]
    
    # Fix issue with 'Audience Type Name': 'CRM-1PD\xa0CRM'
    df['Audience Type Name'] = df['Audience Type Name'].str.replace('CRM-1PD\xa0CRM', 'CRM-1PD CRM')

    # Group dimensions
    if value_column == 'Segment ID':
        df_lvl1 = df.groupby(['Audience Type','Display Funnel Mapped'])[value_column].nunique().reset_index()
    else:
        df_lvl1 = df.groupby(['Audience Type','Display Funnel Mapped'])[value_column].sum().reset_index()

    # Rename columns to Source, Target & Value
    df_lvl1.rename(columns={'Audience Type':'Source', 'Display Funnel Mapped':'Target', value_column:'Value'}, inplace=True)

    # Contact the dataframes in just one
    concat_df = pd.concat([
                           df_lvl1[['Source','Target','Value']]
                        ], ignore_index=True)

    unique_values = pd.unique(concat_df[['Source', 'Target']].values.ravel('K'))
    
    mapping_df = pd.DataFrame({'Label': unique_values.tolist()
                                          , 'Value': [i for i in range(len(unique_values))]})
    
    # Replace values based on the mappings
    concat_df['Source'] = concat_df['Source'].replace(mapping_df.set_index('Label')['Value'])
    concat_df['Target'] = concat_df['Target'].replace(mapping_df.set_index('Label')['Value'])

    
    ##########
    df_source = concat_df.groupby('Source')['Value'].sum().reset_index()
    total_sum_source = df_source['Value'].sum()
    df_source['Percentage'] = (df_source['Value'] / total_sum_source) * 100
    df_source.rename(columns={'Value':'Total','Source':'Value'}, inplace=True)

    df_target = concat_df.groupby('Target')['Value'].sum().reset_index()
    total_sum_target = df_target['Value'].sum()
    df_target['Percentage'] = (df_target['Value'] / total_sum_target) * 100
    df_target.rename(columns={'Value':'Total','Target':'Value'}, inplace=True)

    totals_df = pd.concat([df_source, df_target])

    merged_df = mapping_df.merge(totals_df, on='Value', how='left')

    merged_df['Label_x'] = merged_df['Label'] + "<br>" + merged_df['Percentage'].apply(lambda x: f'{x:.1f}%') + " (" + merged_df['Total'].apply(format_value) + ")"
    ############
    
    
    # Sample data
    source = concat_df['Source'].values.tolist()
    target = concat_df['Target'].values.tolist()
    value = concat_df['Value'].values.tolist()
    labels = merged_df['Label_x'].values.tolist()

    # List of colors for each link based on some condition or data
    colors = generate_gray_tones(len(source))

    # Create links
    link = dict(source=source, target=target, value=value, color=colors)

    # Create nodes
    node = dict(label=labels, pad=30, thickness=20)

    # Create a Sankey object
    chart = go.Sankey(link=link, node=node, arrangement="snap")

    # Build a figure
    fig = go.Figure(chart)
    
    # Add a title to the figure
    fig.update_layout(title_text=title)

    fig.show()
    
    #html_file_path = 'html/' + title + ".html"
    #pio.write_html(fig, file=html_file_path)


In [ ]:
def funnel_vehicle_sankey_plot(df, value_column, by_Dell, title):
    
    if by_Dell:
        df = df[(df['Segment ID'] != 0) & (df['Segment ID'] != '0')]
    
    # Fix issue with 'Audience Type Name': 'CRM-1PD\xa0CRM'
    df['Audience Type Name'] = df['Audience Type Name'].str.replace('CRM-1PD\xa0CRM', 'CRM-1PD CRM')

    # Group dimensions
    if value_column == 'Segment ID':
        df_lvl1 = df.groupby(['Audience Type','Display Funnel Mapped'])[value_column].nunique().reset_index()
        df_lvl2 = df.groupby(['Audience Type','Display Funnel Mapped','Display Dell Vehicle Mapped'])[value_column].nunique().reset_index()
    else:
        df_lvl1 = df.groupby(['Audience Type','Display Funnel Mapped'])[value_column].sum().reset_index()
        df_lvl2 = df.groupby(['Audience Type','Display Funnel Mapped','Display Dell Vehicle Mapped'])[value_column].sum().reset_index()
    
    df_lvl1['Level'] = 1
    df_lvl2['Level'] = 2
    
    # Rename columns to Source, Target & Value
    df_lvl1.rename(columns={'Audience Type':'Source', 'Display Funnel Mapped':'Target', value_column:'Value'}, inplace=True)
    df_lvl2.rename(columns={'Display Funnel Mapped':'Source', 'Display Dell Vehicle Mapped':'Target', value_column:'Value'}, inplace=True)

    # Contact the dataframes in just one
    concat_df = pd.concat([
                           df_lvl1[['Source','Target','Value']]
                           , df_lvl2[['Source','Target','Value']]
                        ], ignore_index=True)

    unique_values = pd.unique(concat_df[['Source', 'Target']].values.ravel('K'))
    
    mapping_df = pd.DataFrame({'Label': unique_values.tolist()
                                          , 'Value': [i for i in range(len(unique_values))]})
    
    # Replace values based on the mappings
    concat_df['Source'] = concat_df['Source'].replace(mapping_df.set_index('Label')['Value'])
    concat_df['Target'] = concat_df['Target'].replace(mapping_df.set_index('Label')['Value'])

        
    ##########
    level_list = concat_df['Level'].unique().tolist()

    if len(level_list) > 1:
        level_list_source = level_list[:-1]
        level_list_target = [level_list[-1]]  # Wrap the last level in a list
    else:
        level_list_source, level_list_target = [1], [1]


    df_source = concat_df[concat_df['Level'].isin(level_list_source)].groupby(['Source','Level'])['Value'].sum().reset_index()
    total_sum_source = df_source[df_source['Level'] == level_list[0]]['Value'].sum()
    df_source['Percentage'] = (df_source['Value'] / total_sum_source) * 100

    df_target = concat_df[concat_df['Level'].isin(level_list_target)].groupby(['Target','Level'])['Value'].sum().reset_index()
    total_sum_source = df_target[df_target['Level'].isin(level_list_target)]['Value'].sum()
    df_target['Percentage'] = (df_target['Value'] / total_sum_source) * 100


    df_source.rename(columns={'Value':'Total','Source':'Value'}, inplace=True)
    df_target.rename(columns={'Value':'Total','Target':'Value'}, inplace=True)

    totals_df = pd.concat([df_source, df_target])

    merged_df = mapping_df.merge(totals_df, on='Value', how='left')

    merged_df['Label_x'] = merged_df['Label'] + "<br>" + merged_df['Percentage'].apply(lambda x: f'{x:.1f}%') + " (" + merged_df['Total'].apply(format_value) + ")"
    ############
    
    
    # Sample data
    source = concat_df['Source'].values.tolist()
    target = concat_df['Target'].values.tolist()
    value = concat_df['Value'].values.tolist()
    labels = merged_df['Label_x'].values.tolist()

    # List of colors for each link based on some condition or data
    colors = generate_gray_tones(len(source))

    # Create links
    link = dict(source=source, target=target, value=value, color=colors)

    # Create nodes
    node = dict(label=labels, pad=30, thickness=20)

    # Create a Sankey object
    chart = go.Sankey(link=link, node=node, arrangement="snap")

    # Build a figure
    fig = go.Figure(chart)
    
    # Add a title to the figure
    fig.update_layout(title_text=title)

    fig.show()
    
    #html_file_path = 'html/' + title + ".html"
    #pio.write_html(fig, file=html_file_path)


In [614]:
def activation_sankey_plot(df, value_column, by_Dell, title):
    
    if by_Dell:
        df = df[(df['Segment ID'] != 0) & (df['Segment ID'] != '0')]
    
    # Fix issue with 'Audience Type Name': 'CRM-1PD\xa0CRM'
    df['Audience Type Name'] = df['Audience Type Name'].str.replace('CRM-1PD\xa0CRM', 'CRM-1PD CRM')

    # Group dimensions
    if value_column == 'Segment ID':
        df_lvl1 = df.groupby(['Audience Type','Audience Type Name'])[value_column].nunique().reset_index()
        df_lvl2 = df.groupby(['Audience Type','Audience Type Name','Audience Source'])[value_column].nunique().reset_index()
        df_lvl3 = df.groupby(['Audience Type','Audience Type Name','Audience Source','Display Dell Vehicle Mapped'])[value_column].nunique().reset_index()
    else:
        df_lvl1 = df.groupby(['Audience Type','Audience Type Name'])[value_column].sum().reset_index()
        df_lvl2 = df.groupby(['Audience Type','Audience Type Name','Audience Source'])[value_column].sum().reset_index()
        df_lvl3 = df.groupby(['Audience Type','Audience Type Name','Audience Source','Display Dell Vehicle Mapped'])[value_column].sum().reset_index()
    
    df_lvl1['Level'] = 1
    df_lvl2['Level'] = 2
    df_lvl3['Level'] = 3
    
    # Rename columns to Source, Target & Value
    df_lvl1.rename(columns={'Audience Type':'Source', 'Audience Type Name':'Target', value_column:'Value'}, inplace=True)
    df_lvl2.rename(columns={'Audience Type Name':'Source', 'Audience Source':'Target', value_column:'Value'}, inplace=True)
    df_lvl3.rename(columns={'Audience Source':'Source', 'Display Dell Vehicle Mapped':'Target', value_column:'Value'}, inplace=True)

    # Contact the dataframes in just one
    concat_df = pd.concat([
                           df_lvl1[['Source','Target','Value', 'Level']]
                           , df_lvl2[['Source','Target','Value', 'Level']]
                           , df_lvl3[['Source','Target','Value', 'Level']]
                        ], ignore_index=True)

    unique_values = pd.unique(concat_df[['Source', 'Target']].values.ravel('K'))
    
    mapping_df = pd.DataFrame({'Label': unique_values.tolist()
                                          , 'Value': [i for i in range(len(unique_values))]})
    
    # Replace values based on the mappings
    concat_df['Source'] = concat_df['Source'].replace(mapping_df.set_index('Label')['Value'])
    concat_df['Target'] = concat_df['Target'].replace(mapping_df.set_index('Label')['Value'])

    
    ##########
    level_list = concat_df['Level'].unique().tolist()

    if len(level_list) > 1:
        level_list_source = level_list[:-1]
        level_list_target = [level_list[-1]]  # Wrap the last level in a list
    else:
        level_list_source, level_list_target = [1], [1]


    df_source = concat_df[concat_df['Level'].isin(level_list_source)].groupby(['Source','Level'])['Value'].sum().reset_index()
    total_sum_source = df_source[df_source['Level'] == level_list[0]]['Value'].sum()
    df_source['Percentage'] = (df_source['Value'] / total_sum_source) * 100

    df_target = concat_df[concat_df['Level'].isin(level_list_target)].groupby(['Target','Level'])['Value'].sum().reset_index()
    total_sum_source = df_target[df_target['Level'].isin(level_list_target)]['Value'].sum()
    df_target['Percentage'] = (df_target['Value'] / total_sum_source) * 100


    df_source.rename(columns={'Value':'Total','Source':'Value'}, inplace=True)
    df_target.rename(columns={'Value':'Total','Target':'Value'}, inplace=True)

    totals_df = pd.concat([df_source, df_target])

    merged_df = mapping_df.merge(totals_df, on='Value', how='left')

    merged_df['Label_x'] = merged_df['Label'] + "<br>" + merged_df['Percentage'].apply(lambda x: f'{x:.1f}%') + " (" + merged_df['Total'].apply(format_value) + ")"
    ############
    
    
    
    
    # Sample data
    source = concat_df['Source'].values.tolist()
    target = concat_df['Target'].values.tolist()
    value = concat_df['Value'].values.tolist()
    labels = mapping_df['Label'].values.tolist()

    # List of colors for each link based on some condition or data
    colors = generate_gray_tones(len(source))

    # Create links
    link = dict(source=source, target=target, value=value, color=colors)

    # Create nodes
    node = dict(label=labels, pad=30, thickness=20)

    # Create a Sankey object
    chart = go.Sankey(link=link, node=node, arrangement="snap")

    # Build a figure
    fig = go.Figure(chart)
    
    # Add a title to the figure
    fig.update_layout(title_text=title)

    fig.show()
    
    #html_file_path = 'html/' + title + ".html"
    #pio.write_html(fig, file=html_file_path)

<h2>Main</h2>

In [615]:
folder_path = r'C:\Users\Rafael_Fagundes\Downloads\compiled_data.csv'

df = pd.read_csv(folder_path, encoding='utf-8')

<h2>DataFrames</h2>

In [616]:
df = df[(df['Fiscal Quarter'] == '2024-Q3')
       & (df['Country_x'] == 'United States')]

csb_df = df[(df['BU'] == 'CSB') 
            & (df['Fiscal Quarter'] == '2024-Q3') 
            & (df['Country_x'] == 'United States')]

b2b_df = df[(df['BU'] == 'B2B') 
            & (df['Fiscal Quarter'] == '2024-Q3') 
            & (df['Country_x'] == 'United States')]

In [617]:
#Spend or Net Rev or Segment ID

<h2>Bu | Type | Vehicle (by Dell)</h2>

In [618]:
bu_audiencetype_vehicle_sankey_plot(df,'Segment ID',True, 'BU and Type and Vehicles - Audiences')
bu_audiencetype_vehicle_sankey_plot(df,'Spend',True, 'BU and Type and Vehicles - Spend')
bu_audiencetype_vehicle_sankey_plot(df,'Net Rev', True, 'BU and Type and Vehicles - Revenue')

<h2>Types | Vehicles</h2>

In [ ]:
print('Segment ID may not be unique as it can be activated in more than one Vehicle')
audiencetype_vehicles_sankey_plot(csb_df, 'Segment ID', True, 'CSB Type and Vehicles - Audiences')
audiencetype_vehicles_sankey_plot(csb_df, 'Spend', False, 'CSB Type and Vehicles - Spend')
audiencetype_vehicles_sankey_plot(csb_df, 'Net Rev', False,'CSB Type and Vehicles - Revenue')

print('Segment ID may not be unique as it can be activated in more than one Vehicle')
audiencetype_vehicles_sankey_plot(b2b_df, 'Segment ID', False, 'B2B Type and Vehicles - Audiences')
audiencetype_vehicles_sankey_plot(b2b_df, 'Spend', False, 'B2B Type and Vehicles - Spend')
audiencetype_vehicles_sankey_plot(b2b_df, 'Net Rev', False, 'B2B Type and Vehicles - Revenue')

<h2>Activation Paths (Type | Type Name | Source | Vehicle)</h2>

In [ ]:
print('Segment ID may not be unique as it can be activated in more than one Vehicle')
activation_sankey_plot(csb_df, 'Segment ID', True, 'CSB Activation Path - Audiences')
activation_sankey_plot(csb_df, 'Spend', True, 'CSB Activation Path - Spend')
activation_sankey_plot(csb_df, 'Net Rev', True, 'CSB Activation Path - Revenue')

print('Segment ID may not be unique as it can be activated in more than one Vehicle')
activation_sankey_plot(b2b_df, 'Segment ID', True, 'B2B Activation Path - Audiences')
activation_sankey_plot(b2b_df, 'Spend', True,'B2B Activation Path - Spend')
activation_sankey_plot(b2b_df, 'Net Rev', True,'B2B Activation Path - Revenue')

In [ ]:
funnel_sankey_plot(csb_df, 'Segment ID', True, 'CSB Funnel - Audiences')
funnel_sankey_plot(csb_df, 'Spend', False, 'CSB Funnel - Spend')
funnel_sankey_plot(csb_df, 'Net Rev', False,'CSB Funnel - Revenue')


funnel_sankey_plot(b2b_df, 'Segment ID', True, 'B2B Funnel - Audiences')
funnel_sankey_plot(b2b_df, 'Spend', False, 'B2B Funnel - Spend')
funnel_sankey_plot(b2b_df, 'Net Rev', False,'B2B Funnel - Revenue')